In [1]:
from google.colab import drive
import os
def prepare():
  !nvidia-smi
  !pip install mxnet-cu100 gluoncv

  DIR = 'a-res_colab'

  drive.mount('/content/drive')
  os.chdir('/content/drive/My Drive/' + DIR)
  if not os.path.exists('/content/SBU-shadow'):
    !unzip data/SBU-shadow.zip -d /content/SBU-shadow
    # ! cp -r data/SBU-shadow/ /content/SBU-shadow
  print('data cpoy to /content/SBU-shadow')

  !ls

prepare()

Tue Apr 28 07:09:28 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P0    32W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
import ipynb_importer, sys
import utils, data_operation as DO
import numpy as np
import mxnet as mx
import time, cv2, gluoncv
from mxnet import gluon, image, nd, init, autograd
from mxnet.gluon import data, model_zoo, nn, loss, utils as gutils
from multiprocessing import cpu_count
import matplotlib.pyplot as plt

importing Jupyter notebook from data_operation.ipynb


In [3]:
CPU_COUNT = cpu_count()
CROP_SIZE = (320, 320)
DATA_DIR = '/content/SBU-shadow/SBU-shadow'
BATCH_SIZE = 16
mx.random.seed(32)

# get train/test dataset object
shadow_train = DO.ShadowDataset(True, CROP_SIZE, DATA_DIR, verbose=True)
shadow_test = DO.ShadowDataset(False, CROP_SIZE, DATA_DIR, verbose=True)

# get train/test iterator
train_iter = data.DataLoader(shadow_train, BATCH_SIZE, shuffle=True,
                last_batch='discard', num_workers=CPU_COUNT)
test_iter = data.DataLoader(shadow_test, BATCH_SIZE, last_batch='discard', 
                            num_workers=CPU_COUNT)

Get 4085 train image, 638 test image.
0 images loaded...
500 images loaded...
1000 images loaded...
1500 images loaded...
2000 images loaded...
2500 images loaded...
3000 images loaded...
3500 images loaded...
4000 images loaded...
Loaded all 4085 features and labels on train dataset.
Remain 3416 images can be cropped and normalized.
Get 4085 train image, 638 test image.
0 images loaded...
500 images loaded...
Loaded all 638 features and labels on test dataset.
Remain 526 images can be cropped and normalized.


In [4]:
import ipynb_importer
import mxnet as mx
from mxnet.gluon import model_zoo, nn
import model.resnet as resnet, model.resnet_attention as resnet_attention, \
  model.resnet_attention_merge as resnet_attention_merge,\
  model.resnet_bcam as resnet_bcam

def new_res18(num_classes = 2):
  # use new resnet
  net = resnet.Resnet_18()[:-2]
  num_classes = num_classes
  net.add(nn.Conv2D(num_classes, kernel_size=1),
          nn.Conv2DTranspose(num_classes, kernel_size=64, padding=16,
                            strides=32))
  return net


def res18_attention(num_classes = 2):
  # use new resnet
  net = resnet_attention.Resnet_18()[:-2]
  num_classes = num_classes
  net.add(nn.Conv2D(num_classes, kernel_size=1),
          nn.Conv2DTranspose(num_classes, kernel_size=64, padding=16,
                            strides=32))
  return net


def res18_attention_merge(num_classes = 2):
  # use new resnet
  net = resnet_attention_merge.Resnet_18()[:-2]
  num_classes = num_classes
  net.add(nn.Conv2D(num_classes, kernel_size=1),
          nn.Conv2DTranspose(num_classes, kernel_size=64, padding=16,
                            strides=32))
  return net


def res34_attention(num_classes = 2):
  # use new resnet
  net = resnet_attention.Resnet_34()[:-2]
  num_classes = 2
  net.add(nn.Conv2D(num_classes, kernel_size=1),
          nn.Conv2DTranspose(num_classes, kernel_size=64, padding=16,
                            strides=32))
  return net


def pretrained_res18(num_classes = 2):
  # use pretrained resnet
  pretrained_net = model_zoo.vision.resnet18_v2(pretrained=True)
  net = nn.HybridSequential()
  # drop global_avgpool, flatten layer
  for layer in pretrained_net.features[:-2]:
    net.add(layer)

  # add 1*1_conv change channel, conv_transpose upsample image
  
  net.add(nn.Conv2D(channels=num_classes, kernel_size=1),
          nn.Conv2DTranspose(channels=num_classes, kernel_size=64, padding=16,
                            strides=32))
  return net
  
def pretrained_res50(num_classes = 2):
  # use pretrained resnet
  pretrained_net = model_zoo.vision.resnet50_v2(pretrained=True)
  net = nn.HybridSequential()
  # drop global_avgpool, flatten layer
  for layer in pretrained_net.features[:-2]:
    net.add(layer)

  # add 1*1_conv change channel, conv_transpose upsample image
  
  net.add(nn.Conv2D(channels=num_classes, kernel_size=1),
          nn.Conv2DTranspose(channels=num_classes, kernel_size=64, padding=16,
                            strides=32))
  
  return net

importing Jupyter notebook from /content/drive/My Drive/a-res_colab/model/resnet.ipynb
importing Jupyter notebook from /content/drive/My Drive/a-res_colab/model/resnet_attention.ipynb
importing Jupyter notebook from /content/drive/My Drive/a-res_colab/model/resnet_attention_merge.ipynb
importing Jupyter notebook from /content/drive/My Drive/a-res_colab/model/resnet_bcam.ipynb


In [0]:
def dice_loss(y_pred, y_true, batch_axis=0):
  smooth = 1.
  # y_pred = y_pred.log_softmax(y_pred, axis=1)
  y_pred = nd.sigmoid(y_pred.reshape(y_pred.shape[0], y_pred.shape[2], y_pred.shape[3]))
  # y_pred = (y_pred > benchmark)
  # classes = y_pred.shape[1]
  # for i in range(classes):
  #   product = nd.multiply(y_pred[:,i,:,:], y_true)
  #   intersection = nd.sum(product)
  #   coefficient = (2.*intersection + smooth) / (nd.sum(y_pred[:,i,:,:])+nd.sum(y_true) +smooth)
  #   loss = 1. - coefficient
  #   # or "-coeficient"
  #   loss_total = loss_total + loss
  product = nd.multiply(y_pred, y_true)
  intersection = nd.sum(product)
  coefficient = (2.*intersection + smooth) / (nd.sum(y_pred) + nd.sum(y_true) + smooth)
  loss = 1. - coefficient
  # or "-coefficient"
  return loss

def balanced_SBCE(y_pred, y_true, batch_axis=0, w=0.15):
  y_pred = nd.sigmoid(nd.relu(y_pred.reshape(y_pred.shape[0], y_pred.shape[2], y_pred.shape[3])))
  loss = -(w*y_true*nd.log(y_pred+1e-12) + (1.-w)*(1.-y_true)*nd.log(1.-y_pred+1e-12))
  return nd.mean(loss, axis=batch_axis, exclude=True)
  # return loss


def focal_loss(y_pred, y_true, batch_axis=0, w=0.2, gamma=2):
  y_pred = nd.sigmoid(nd.relu(y_pred.reshape(y_pred.shape[0], y_pred.shape[2], y_pred.shape[3])))
  loss = -(w*y_true*((1.-y_pred)**gamma)*nd.log(y_pred+1e-12) + (1.-w)*(1.-y_true)*(y_pred**gamma)*nd.log(1.-y_pred+1e-12))
  return nd.mean(loss, axis=batch_axis, exclude=True)
  # return nd.sum(loss)/y_pred.size
  # return loss


def bilinear_kernel(in_channels, out_channels, kernel_size):
  factor = (kernel_size + 1) // 2
  if kernel_size % 2 == 1:
      center = factor - 1
  else:
      center = factor - 0.5
  og = np.ogrid[:kernel_size, :kernel_size]
  filt = (1 - abs(og[0] - center) / factor) * \
          (1 - abs(og[1] - center) / factor)
  weight = np.zeros((in_channels, out_channels, kernel_size, kernel_size),
                    dtype='float32')
  weight[range(in_channels), range(out_channels), :, :] = filt
  return nd.array(weight)


def shadow_evaluate(img1, img2, benchmark=0.5):
  SER, NER, BER, Acc = 0.0, 0.0, 0.0, 0.0
  Tp, Tn, Np, Nn = 0, 0, 0, 0
  # img1 = img1.reshape(img1.shape[0], img1.shape[1], img1.shape[2])
  assert img1.size == img2.size, 'two img with different shape {} -> {}'.format(img1.shape, img2.shape)
  # img1 = nd.sigmoid(img1)
  # img1 = (img1 > benchmark)
  # print(img1)
  img1 = img1 * 1.0
  img2 = img2 * 1.0
  Nn = (img2 == 0).sum().asscalar()
  Np = img2.size - Nn
  # a=1,b=1,a+b=2 both shadow
  # a=0,b=0,a+b=0 both background
  # a=1,b=0 or a=0,b=1, a+b=1, feature different with label
  Tp = (img1 + img2 == 2).sum().asscalar()
  Tn = (img1 + img2 == 0).sum().asscalar()
  # SER += (1 - (Tp / Np)) * 100
  # NER += (1 - (Tn / Nn)) * 100
  # BER += (SER + NER) / 2
  # Acc += (Tp + Tn) / (Np + Nn)
  # print(Tp, Tn, Np, Nn)
  return Tp, Tn, Np, Nn


def train(train_iter, test_iter, net, optimizer, ctx, num_epochs, lr, 
          lr_period, lr_decay, wd, model_name, loss, checkpoint_period=20, retrain=0, retrain_data=0):
  plot_train_loss = []
  plot_train_acc = []
  plot_test_acc = []
  data_time = time.strftime('%Y_%m_%d_%H_%M_%S') if retrain_data == 0 else retrain_data

  export_path = os.path.join('./param', model_name, data_time)

  trainer = gluon.Trainer(net.collect_params('.'), optimizer, {'learning_rate': lr, 'wd': wd, 'momentum':0.9})
  # trainer = gluon.Trainer(net.collect_params(), optimizer, {'learning_rate': lr, 'wd': wd})
  if isinstance(ctx, mx.Context):
    ctx = [ctx]
  print('Starting training on ', ctx)

  for epoch in range(retrain+1, num_epochs):
    # learning rate decay
    if epoch > 0 and epoch % lr_period == 0:
      print('learning_rate: ', trainer.learning_rate, ' -> ', trainer.learning_rate * lr_decay)
      trainer.set_learning_rate(trainer.learning_rate * lr_decay)

    train_loss_sum, train_acc_sum, n, m, start = 0.0, 0.0, 0, 0, time.time()
    # evaluate train_data
    for i, batch in enumerate(train_iter):
      features, labels = batch
      if labels.dtype != features.dtype:
        labels = labels.astype(features.dtype)
      Xs = gutils.split_and_load(features, ctx)
      ys = gutils.split_and_load(labels, ctx)
      batch_size = features.shape[0]
      ls = []
      with autograd.record():
        y_hats = [net(X) for X in Xs]
        ls = [loss(y_hat, y) for y_hat, y in zip(y_hats, ys)]
      for l in ls:
        l.backward()
      trainer.step(batch_size)
      train_loss_sum += sum([l.sum().asscalar() for l in ls])
      n += sum([l.size for l in ls])
      train_acc_sum += sum([((y_hat.argmax(axis=1)) == y).sum().asscalar() for y_hat, y in zip(y_hats, ys)])
      # train_acc_sum += sum([((y_hat.reshape(y_hat.shape[0],y_hat.shape[2],y_hat.shape[3])>0.5) == y).sum().asscalar() for y_hat, y in zip(y_hats, ys)])
      m += sum([y.size for y in ys])
      
    # evaluate test_data
    test_acc_sum, test_sample_sum = 0.0, 0
    test_Tp_sum, test_Tn_sum, test_Np_sum, test_Nn_sum = 0, 0, 0, 0
    for _, test_batch in enumerate(test_iter):
      features, labels = test_batch
      if labels.dtype != features.dtype:
        labels = labels.astype(features.dtype)
      Xs = gutils.split_and_load(features, ctx)
      ys = gutils.split_and_load(labels, ctx)
      for X, y in zip(Xs, ys):
        Tp, Tn, Np, Nn = shadow_evaluate(net(X).argmax(axis=1), y)
        # Tp, Tn, Np, Nn = shadow_evaluate(net(X).reshape(X.shape[0],X.shape[2],X.shape[3]), y)
        # test_acc_sum += (net(X).argmax(axis=1) == y).sum().copyto(mx.cpu())
        test_Tp_sum += Tp
        test_Tn_sum += Tn
        test_Np_sum += Np
        test_Nn_sum += Nn

        nd.waitall()
        # test_sample_sum += y.size
      # test_Tp_sum.wait_to_read()
      # test_Tn_sum.wait_to_read()
      # test_Np_sum.wait_to_read()
      # test_Nn_sum.wait_to_read()
    # test_SER = test_SER_sum / test_sample_sum
    # test_NER = test_NER_sum / test_sample_sum
    # test_BER = test_BER_sum / test_sample_sum
    # test_acc = test_acc_sum / test_sample_sum
    test_SER = (1 - (test_Tp_sum / test_Np_sum)) * 100
    test_NER = (1 - (test_Tn_sum / test_Nn_sum)) * 100
    test_BER = (test_SER + test_NER) / 2
    test_Acc = (test_Tp_sum + test_Tn_sum) / (test_Np_sum + test_Nn_sum)
    print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, SER/NER/BER %.2f/%.2f/%.2f, time %.1f sec'
    % (epoch, train_loss_sum / n, train_acc_sum / m, test_Acc, 
       test_SER, test_NER, test_BER, time.time() - start))
    

    # export params per 20 epochs
    if epoch % checkpoint_period == 0:
      if not os.path.exists(export_path):
        os.makedirs(export_path)
      # net.export(path=export_path, epoch=epoch)
      params_path = model_name + ('_%d_%.4f_%.3f_%.2f.params' 
                % (epoch, train_loss_sum / n, train_acc_sum / m, test_BER))
      save_path = os.path.join(export_path, params_path)
      print('params save at', save_path)
      net.save_parameters(save_path)

  # record loss and save as png
    plot_train_loss.append(train_loss_sum / n)
    plot_train_acc.append(train_acc_sum / m)
    plot_test_acc.append(test_Acc)
  plt.plot(plot_train_loss)
  plt.plot(plot_train_acc)
  plt.plot(plot_test_acc)
  plt.legend(['train_loss', 'train_acc', 'test_acc'])
  plt.savefig(os.path.join(export_path, 'training_loss.png'))

  if not os.path.exists(export_path):
    os.makedirs(export_path)
    # net.export(path=export_path, epoch=epoch)
    params_path = model_name + ('_%d_%.4f_%.3f_%.2f.params' 
              % (epoch, train_loss_sum / n, train_acc_sum / m, test_BER))
    save_path = os.path.join(export_path, params_path)
    print('params save at', save_path)
    net.save_parameters(save_path)
  

In [0]:
##########
## retrain
##########
def load_model(model, model_name, retrain_step, start_train=False):
  net = model

  import_path = './param'
  print(os.listdir(import_path))
  model_path = str(input('select model path: '))
  import_path = os.path.join(import_path, model_path)
  print(sorted(os.listdir(import_path), reverse = True))
  data_path = str(input('select data path: '))
  import_path = os.path.join(import_path, data_path)
  print(os.listdir(import_path)[-5:])
  file_path = str(input('select file path: '))
  import_path = os.path.join(import_path, file_path)
  
  net.load_parameters(import_path)
  ctx = utils.try_all_gpus()
  net.collect_params().reset_ctx(ctx)
  print('load finished on ', ctx)

  if start_train:
    retrain = int(file_path.split('_')[-4])
    train_loss = balSoftmaxCrossEntropyLoss(axis=1)
    lr_period = 25
    lr_decay = 0.25
    lr = 0.01
    step = (retrain + 1) // lr_period
    print('continue training at epoch ', retrain, ' target ', retrain + retrain_step)
    # train(train_iter, test_iter, net, 'sgd', ctx=ctx, num_epochs=retrain+retrain_step, 
    #       lr=lr*(lr_decay**step), lr_period=lr_period, lr_decay=lr_decay, 
    #       wd=1e-3, model_name=model_name, checkpoint_period=10, loss=train_loss, retrain=retrain, retrain_data=data_path)
    train(train_iter, test_iter, net, 'sgd', ctx=ctx, num_epochs=retrain+retrain_step, 
          lr=0.004, lr_period=lr_period, lr_decay=lr_decay, 
          wd=1e-3, model_name=model_name, checkpoint_period=25, loss=train_loss, retrain=retrain, retrain_data=data_path)
  return net
    

def predict(img, net):
  X = test_iter._dataset.normalize_image(img)
  X = X.transpose((2, 0, 1)).expand_dims(axis=0)
  pred = nd.argmax(net(X.as_in_context(ctx[0])), axis=1)
  # pred = net(X.as_in_context(ctx[0]))>0.5
  return pred.reshape((X.shape[2], X.shape[3]))

In [0]:
class FocalLoss(gluon.loss.Loss):
  def __init__(self, axis=1, alpha=0.2, gamma=0, batch_axis=0, **kwargs):
    super(FocalLoss, self).__init__(None, batch_axis, **kwargs)
    self._axis = axis
    self._alpha = alpha
    self._gamma = gamma

  def hybrid_forward(self, F, output, label):
    output = F.softmax(output)
    pj = F.pick(output, label, axis=self._axis, keepdims=True)
    loss = - (self._alpha * ((1 - pj) ** self._gamma) * pj.log())
    return F.mean(loss, axis=self._batch_axis, exclude=True)


class balSoftmaxCrossEntropyLoss(gluon.loss.Loss):
    def __init__(self, axis=-1, sparse_label=True, from_logits=False, weight=None,
              batch_axis=0, **kwargs):
      super(balSoftmaxCrossEntropyLoss, self).__init__(
          weight, batch_axis, **kwargs)
      self._axis = axis
      self._sparse_label = sparse_label
      self._from_logits = from_logits

    def hybrid_forward(self, F, pred, label, sample_weight=None):
      if not self._from_logits:
        pred = F.log_softmax(pred, self._axis)
        a = F.slice_axis(pred, axis=1, begin=0, end=1)
        b = F.slice_axis(pred, axis=1, begin=1, end=2)*10
        pred = F.concat(a, b)
      if self._sparse_label:
        loss = -F.pick(pred, label, axis=self._axis, keepdims=True)
      else:
        label = _reshape_like(F, label, pred)
        loss = -F.sum(pred * label, axis=self._axis, keepdims=True)
      # loss = _apply_weighting(F, loss, self._weight, sample_weight)
      return F.mean(loss, axis=self._batch_axis, exclude=True)

# class BERLoss(gluon.loss.Loss):
#     def __init__(self, axis=-1, sparse_label=True, from_logits=False, weight=None,
#               batch_axis=0, **kwargs):
#       super(BERLoss, self).__init__(
#           weight, batch_axis, **kwargs)
#       self._axis = axis
#       self._sparse_label = sparse_label
#       self._from_logits = from_logits

#     def hybrid_forward(self, F, pred, label, sample_weight=None):
#       pred = pred.argmax(axis=1)
#       Tp, Tn, Np, Nn = shadow_evaluate(pred, label)
#       L1 = -(Nn/(Np+Nn))*label*(pred+1e-12).log() - (Np/(Np+Nn))*(1-label)*(1-pred+1e-12).log()
#       L2 = -(1-(Tp/Np))*label*(pred+1e-12).log() - (1-(Tn/Nn))*(1-label)*(1-pred+1e-12).log()
#       loss = F.sum(L1 + L2, keepdims=True)/pred.size
#       # return F.mean(loss, axis=self._batch_axis, exclude=True)
#       return loss

class FocalLoss(gluon.loss.Loss):
  def __init__(self, axis=-1, alpha=0.25, gamma=2, sparse_label=True,
                 from_logits=False, batch_axis=0, weight=None, num_class=None,
                 eps=1e-12, size_average=True, **kwargs):
        super(FocalLoss, self).__init__(weight, batch_axis, **kwargs)
        self._axis = axis
        self._alpha = alpha
        self._gamma = gamma
        self._sparse_label = sparse_label
        if sparse_label and (not isinstance(num_class, int) or (num_class < 1)):
            raise ValueError("Number of class > 0 must be provided if sparse label is used.")
        self._num_class = num_class
        self._from_logits = from_logits
        self._eps = eps
        self._size_average = size_average

  def hybrid_forward(self, F, pred, label, sample_weight=None):
        """Loss forward"""
        if not self._from_logits:
            pred = F.sigmoid(pred)
        if self._sparse_label:
            one_hot = F.one_hot(label, self._num_class)
        else:
            one_hot = label > 0
        one_hot = one_hot.transpose((0, 3, 1, 2)) 
        pt = F.where(one_hot, pred, 1 - pred)
        t = F.ones_like(one_hot)
        alpha = F.where(one_hot, self._alpha * t, (1 - self._alpha) * t)
        loss = -alpha * ((1 - pt) ** self._gamma) * F.log(F.minimum(pt + self._eps, 1))
        # loss = _apply_weighting(F, loss, self._weight, sample_weight)
        if self._size_average:
            return F.mean(loss, axis=self._batch_axis, exclude=True)
        else:
            return F.sum(loss, axis=self._batch_axis, exclude=True)

In [0]:
##########
## train
# ##########
# num_classes = 2
# net = pretrained_res18(num_classes)
# net[:-1].initialize(init=init.Xavier())
# # init net[-1:]
# net.initialize(init=init.Xavier())
# net[-1].initialize(init.Constant(bilinear_kernel(num_classes, num_classes, 64)), force_reinit=True)
# net[-2].initialize(init=init.Normal(sigma=0.01), force_reinit=True)

# train
# train_loss = loss.SigmoidBinaryCrossEntropyLoss()
# train_loss = FocalLoss()
def combine_loss(y_pred, y_true):
  loss1 = train_loss(y_pred, y_true)
  loss2 = dice_loss(y_pred, y_true)
  return loss1 + loss2

model_name = 'res34_bcam_parallel'
# net = resnet_bcam.res50_bcam()
net = resnet_bcam.get_net(model=model_name)
# print(net)
ctx = utils.try_all_gpus()

# train_loss = loss.SoftmaxCrossEntropyLoss(axis=1)
# train_loss = balSoftmaxCrossEntropyLoss(axis=1)
# train_loss = dice_loss()
train_loss = FocalLoss(axis=1, num_class=2)
net.collect_params('.').reset_ctx(ctx)
a = nd.random.uniform(shape=(1,3,320,320), ctx=ctx[0])
net(a)
net.hybridize()
train(train_iter, test_iter, net, 'sgd', ctx=ctx, num_epochs=500, 
    lr=0.01, lr_period=100, lr_decay=0.5, wd=1e-3, model_name=model_name, checkpoint_period=50, loss=train_loss)

# trainer = gluon.Trainer(net.collect_params('.'), 'sgd', {'learning_rate': 0.01, 'wd': 1e-3, 'momentum':0.9})
# utils.train(train_iter, test_iter, net, train_loss, trainer, ctx=ctx, num_epochs=3)

res34_bcam_parallel  loading
res34_bcam_parallel  initialize finished
Starting training on  [gpu(0)]
epoch 1, loss 0.0561, train acc 0.827, test acc 0.839, SER/NER/BER 72.50/0.93/36.71, time 159.1 sec
epoch 2, loss 0.0394, train acc 0.857, test acc 0.867, SER/NER/BER 56.49/1.73/29.11, time 121.1 sec
epoch 3, loss 0.0353, train acc 0.874, test acc 0.838, SER/NER/BER 74.06/0.75/37.40, time 121.1 sec
epoch 4, loss 0.0334, train acc 0.881, test acc 0.882, SER/NER/BER 48.09/2.54/25.32, time 121.2 sec
epoch 5, loss 0.0316, train acc 0.889, test acc 0.876, SER/NER/BER 51.54/1.60/26.57, time 121.1 sec
epoch 6, loss 0.0304, train acc 0.892, test acc 0.881, SER/NER/BER 48.31/1.97/25.14, time 121.1 sec
epoch 7, loss 0.0297, train acc 0.893, test acc 0.896, SER/NER/BER 36.30/3.35/19.83, time 121.2 sec
epoch 8, loss 0.0294, train acc 0.894, test acc 0.894, SER/NER/BER 37.81/3.06/20.44, time 121.1 sec
epoch 9, loss 0.0285, train acc 0.897, test acc 0.876, SER/NER/BER 52.27/1.72/27.00, time 121.2 sec

In [0]:
### retrain ###

model_name = 'res34_bcam_parallel'
net1 = resnet_bcam.get_net(model=model_name)
net1 = load_model(net, model_name, retrain_step=500, start_train=False)

# model_name = 'res34_bcam'
net2 = new_res18()
net2 = load_model(net, model_name, retrain_step=500, start_train=False)

# net3 = resnet_bcam.get_net(model=model_name)
# net3 = load_model(net, model_name, retrain_step=500, start_train=False)

In [0]:
ctx = utils.try_all_gpus()
test_images, test_labels = DO.read_file(data_dir=DATA_DIR, train_file=False)

In [0]:
def show_images(imgs, num_rows, num_cols, scale=2):
    """Plot a list of images."""
    figsize = (num_cols * scale, num_rows * scale)
    _, axes = plt.subplots(num_rows, num_cols, figsize=figsize)
    for i in range(num_rows):
        for j in range(num_cols):
            axes[i][j].imshow(imgs[i * num_cols + j].asnumpy())
            axes[i][j].axes.get_xaxis().set_visible(False)
            axes[i][j].axes.get_yaxis().set_visible(False)
    return axes

n, imgs = 5, []
for i in range(250, 258):
  # not crop
  # X = test_images[i]
  # pred = predict(X)
  # pred = pred.copyto(mx.cpu())
  # pred = cv2.cvtColor(pred.asnumpy(), cv2.COLOR_GRAY2RGB)
  # pred = nd.array(pred)
  # imgs += [X, pred, test_labels[i]]

  # crop
  crop_size = (320, 320)
  X, crop_rect = image.random_crop(test_images[i], crop_size)
  pred1 = predict(X, net1)
  pred1 = cv2.cvtColor(pred1.asnumpy(), cv2.COLOR_GRAY2RGB)
  pred1 = nd.array(pred1)

  pred2 = predict(X, net2)
  pred2 = cv2.cvtColor(pred2.asnumpy(), cv2.COLOR_GRAY2RGB)
  pred2 = nd.array(pred2)
  imgs += [X, pred1, pred2, image.fixed_crop(test_labels[i], *crop_rect)]
axes=show_images(imgs[::4] + imgs[1::4] + imgs[2::4] + imgs[3::4], 4, 8);

256